In [ ]:
#######This code is for automatically calculating and exporting two of the features within 
##the machine learning model to an excel sheet ('friday short volume' and 'friday short volume-5 day average')

## It functions by taking in a list of stock tickers at the stock variable
##it then gets the short volume data from the fintel webpage 
##by digesting the html code and pulling out the relevant information
##for a respective stock
##Fintel has throttling on its website (will block your IP if you make too many requests
##during a given time frame) To deal with this i am making my requests through the 
##application Tor, and using it to change my IP address every request
##In order to use this you need to download the Tor application: https://www.torproject.org/download/
#and then download the torrequest python library and configure one of the files within the 
##torrequest package by adding the tor application executable file path to a line of 
#python code, process described in detail here: https://stackoverflow.com/questions/24801924/tor-with-python-stem-basic-tor-not-in-path
#(the second answer is the one that describes what i did)

#Inputs you need to make:
#1. specify export file path location
#2. specify your stock ticker list
#3. specify dates in your week
#4. specify friday
##This hasnt been completely trouble shot, it will throw a couple of different errors 
## that mostly deal with processing the html code. When that happens the code will export
##'NaN' for the feature export values associated with that ticker
## One error that has been successfully dealt with is 'days missing'. For whatever reason
##stocks sometimes won't have a day of short volume data, i deal with this by counting how many 
##times that this happens and exporting it to the sheet. 
#any stock with more than one day of short data missing and or friday short data missing gets
#doesn't get included in the machine learning model.  
import requests
import re 
import string
import time 
import numpy as np 
import pandas as pd 
import xlsxwriter
from torrequest import TorRequest
import requests
from torpy import TorClient
import os 
import torpy
from torpy.http.requests import TorRequests
from multiprocessing.pool import ThreadPool
from torpy.http.requests import tor_requests_session
from stem.control import Controller
import stem 


torprocess = stem.process.launch_tor(tor_cmd = '/Applications/Tor Browser.app/Contents/Resources/TorBrowser/Tor/tor')
cookies = dict(BCPermissionLevel='PERSONAL')
proxy = {'http':  'socks5://127.0.0.1:9050',
           'https': 'socks5://127.0.0.1:9050'}
   # return session
#################################specify export file location############################################
workbook=xlsxwriter.Workbook('/Users/jpjansen/Documents/Shorts_1_4V2.xlsx')
##############################################################################################
worksheet = workbook.add_worksheet()
worksheet.write(0,0,'Stock Symbol')
worksheet.write(0,1,'Friday Short Ratio')
worksheet.write(0,2,'Friday-Week Average')
worksheet.write(0,3,'Week Average')
worksheet.write(0,4,'Days Missing')
######''#########stock list####################################################################
stock = ['EQOS', 'FUBO', 'IPOC', 'GENE', 'NXTD', 'PHUN', 'ETH', 'EBON', 'DPW', 'NCTY', 'HOFV', 'NETE', 'NGA', 'LAC', 'XL', 'AAL', 'RIOT', 'FARM', 'MARA', 'OCGN', 'BNGO', 'SNDL', 'IDEX', 'PANA', 'THMO', 'PTMN', 'SOAC', 'AIH', 'XSPA', 'MGM', 'SUNW', 'DGLY', 'BTBT', 'JAGX', 'ZOM', 'GT', 'FEYE', 'FLNT', 'MP', 'LGVW', 'VERO', 'ELF', 'AINC', 'SPCE', 'PLUG', 'CCL', 'ACB', 'XCEL', 'HIMX', 'CAN', 'GP', 'ZKIN', 'THCB', 'ACAM']
url = "https://fintel.io/ss/us/"
allow = string.ascii_lowercase[1]+string.digits+"="+"."+"-"
friday_shorts = []
week_average = []
friday_subtractmean = [] 
counter = 0 

for k in range(len(stock)):
    loopURL = url+stock[k]
    with TorRequest(proxy_port=9050, ctrl_port=9051, password = 'password') as tr:
        res = tr.get(loopURL, proxies = proxy, headers={"User-Agent": "Mozilla/5.0"}, cookies=cookies)
        ipaddress= tr.get('http://ipecho.net/plain')
        print ("New Ip Address",ipaddress.text)
        tr.reset_identity()
        tr.close()
    print(stock[k])

    a = res.content
    b = a.decode('UTF-8')
    c = b.replace('"', '')
    c = c.replace("'", '')
    c = c.replace(",", '')
    c = c.replace(";",'')
    c = c.replace(">",'')
    c = c.replace("<",'')
    c = c.replace("/",'')
    c = c.replace("/",'')
    c = c.replace('','')
    c = c.replace("\r","")
    c = c.replace("\n","")
   
    try:
        d = c.split("Short Volume Ratio adivdiv class=col-xs-12 col-sm-12 col-md-12 col-lg-6table id=topic-table-body")
        e = d[1]
        f = e.split('tdtrtbodytabledivdivdivdiv')
    except IndexError:
        friday = 'NaN'
        week_average = 'NaN'
        friday_subtractmean = 'NaN'
        print(stock[k],'INDEX ERROR')
        
        worksheet.write(counter+1,0,stock[k])
        worksheet.write(counter+1,1,friday)
        worksheet.write(counter+1,2,friday_subtractmean)
        worksheet.write(counter+1,3,week_average)
        counter+=1
        time.sleep(1)
        continue 
  
    asdf = re.sub('[^%s]'% allow, '', f[0])
    testing = asdf.split('=')
    data = []
    for i in range(len(testing)):
        z = testing[i].split('b')
        for q in range(len(z)):
            if z[q]!='':
                if z[q]!='-':
                    data.append(z[q])
###############################specify dates in week############################################################                
    previous_week_dates = ['2020-12-28','2020-12-29','2020-12-30','2020-12-31'] 
############################################################################################
    try:  
        dates = np.transpose(data[0::4])
        shortvolume = np.transpose(data[1::4])
        totalvolume = np.transpose(data[2::4])
        shortratio = np.transpose(data[3::4])
        SVfloat = []
        TVfloat = []
        SRfloat = []
        for i in range(len(shortratio)):
            SVfloat.append(np.abs(float(shortvolume[i])))
            TVfloat.append(np.abs(float(totalvolume[i])))
            SRfloat.append(np.abs(float(shortratio[i])))
     
        final_data = np.column_stack((dates, SVfloat, TVfloat, SRfloat))
        df = pd.DataFrame(data=final_data[:,1:], index=[dates], columns=["short volume", "total volume", "short ratio"])
        try:
####################################specify friday date #############################################################
            friday = float(df.loc['2020-12-31','short ratio'].values[0])
#################################################################################################
            friday_shorts.append(friday)
        except KeyError:
            friday = 'NaN'
            week_average = 'NaN'
            friday_subtractmean = 'NaN'
            print(stock[k],'INDEX ERROR')

            worksheet.write(counter+1,0,stock[k])
            worksheet.write(counter+1,1,friday)
            worksheet.write(counter+1,2,friday_subtractmean)
            worksheet.write(counter+1,3,week_average)
            counter+=1
            time.sleep(1)
            continue 
   
        short_ratios = [] 
        days_missing = 0 
        for i in range(len(previous_week_dates)):
            try:
                short_ratios.append(float(df.loc[previous_week_dates[i],'short ratio'].values[0]))
            except KeyError:
                days_missing+=1
        week_average = np.sum(short_ratios)/len(short_ratios)  
        friday_subtractmean = friday-week_average
    
    except ValueError:
        print("Value Error", stock[k],shortvolume[i])
        week_average = 'NaN'
        friday_subtractmean = 'NaN'
    print(df)
    worksheet.write(counter+1,0,stock[k])
    worksheet.write(counter+1,1,friday)
    worksheet.write(counter+1,2,friday_subtractmean)
    worksheet.write(counter+1,3,week_average)
    worksheet.write(counter+1,4,days_missing)
    worksheet.write(counter+1,5,ipaddress.text)

    counter+=1
    time.sleep(1)
workbook.close()